訓練模型

In [560]:
#%%
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Bidirectional
from keras.models import load_model

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import joblib
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import os

#載入訓練資料
device = 'L4'
SourceData = pd.read_csv(f"..//Data//MergedSorted//{device}_Merged_Sorted.csv")
SourceData.dropna(inplace=True)
SourceData.reset_index(inplace=True)

In [561]:
input_features_model_1 = to_predict_features_model_1 = [
    'Avg_Temperature(°C)',
    'Avg_Humidity(%)',
    'Avg_Sunlight(Lux)',
    'Avg_Power(mW)',
    
    'Min_Temperature(°C)',
    'Min_Humidity(%)',
    'Min_Sunlight(Lux)',
    'Min_Power(mW)',
    
    'Max_Temperature(°C)',
    'Max_Humidity(%)',
    'Max_Sunlight(Lux)',
    'Max_Power(mW)'
    
    # 'Std_Avg_Temperature(°C)',
    # 'Std_Avg_Humidity(%)',
    # 'Std_Avg_Sunlight(Lux)',
    # 'Std_Avg_Power(mW)',
    
    # 'Std_Min_Temperature(°C)',
    # 'Std_Min_Humidity(%)',
    # 'Std_Min_Sunlight(Lux)',
    # 'Std_Min_Power(mW)',
    
    # 'Std_Max_Temperature(°C)',
    # 'Std_Max_Humidity(%)',
    # 'Std_Max_Sunlight(Lux)',
    # 'Std_Max_Power(mW)'
]

In [562]:
def create_dataset(data, LookBackNum):
    X = []
    y = []

    #設定每i-12筆資料(X_train)就對應到第i筆資料(y_train)
    for i in range(LookBackNum,len(data)):
        X.append(data[i-LookBackNum:i, :])
        y.append(data[i, :])

    return np.array(X), np.array(y)
  
#設定LSTM往前看的筆數和預測筆數
n_timesteps = LookBackNum = 48 #LSTM往前看的筆數，一筆10分鐘



#LSTM 選擇要留下來的資料欄位
AllOutPut = SourceData[to_predict_features_model_1].values


#正規化
X_preprocess_pipe = make_pipeline(
    StandardScaler(),
    # PolynomialFeatures(degree=2)
)

y_preprocess_pipe = make_pipeline(
    MinMaxScaler()
)

y_preprocess_pipe.fit(AllOutPut)
AllOutPut_y_Scaled = y_preprocess_pipe.transform(AllOutPut)

X_preprocess_pipe.fit(AllOutPut)
AllOutPut_X_Scaled = X_preprocess_pipe.transform(AllOutPut)

X_train, _ = create_dataset(AllOutPut_X_Scaled, LookBackNum=LookBackNum)
_, y_train = create_dataset(AllOutPut_y_Scaled, LookBackNum=LookBackNum)

n_features = X_train.shape[2]
n_predictiton = y_train.shape[1]

# Reshaping
#(samples 是訓練樣本數量,timesteps 是每個樣本的時間步長,features 是每個時間步的特徵數量)
X_train = np.reshape(X_train,(X_train.shape[0], n_timesteps, n_features))
X_train.shape

(5902, 48, 91)

In [565]:

#%%
#============================建置&訓練「LSTM模型」============================
#建置LSTM模型
from tensorflow.keras import backend as K

# 自訂Loss Function
def mae_mse_loss(ratio=0.8):
    def mae_mse(y_true, y_pred):
        mae = K.mean(K.abs(y_true - y_pred), axis=-1)
        mse = K.mean(K.square(y_true - y_pred), axis=-1)
        return ratio * mae + (1 - ratio) * mse
    
    return mae_mse

regressor = Sequential()

### input layer ###
regressor.add(Bidirectional(
    LSTM(units=512, return_sequences=True, activation='relu'), input_shape=(n_timesteps, n_features)))

### hidden layer ###

regressor.add(Dense(units=400))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units=128, activation='relu'))

regressor.add(Dense(units=50))
regressor.add(Dropout(0.1))
# regressor.add(Dropout(0.1))

# regressor.add(LSTM(units=64, activation='tanh'))

### ouput layer ###

regressor.add(Dense(units=n_predictiton)) 

regressor.compile(
    optimizer='adam', 
    loss=mae_mse_loss(ratio=0.1), 
    metrics=['mae','mse']
    )

regressor.summary()

Model: "sequential_91"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_83 (Bidirecti  (None, 48, 1024)         2473984   
 onal)                                                           
                                                                 
 dense_194 (Dense)           (None, 48, 400)           410000    
                                                                 
 dropout_85 (Dropout)        (None, 48, 400)           0         
                                                                 
 lstm_183 (LSTM)             (None, 128)               270848    
                                                                 
 dense_195 (Dense)           (None, 50)                6450      
                                                                 
 dropout_86 (Dropout)        (None, 50)                0         
                                                     

In [564]:
#開始訓練
regressor.fit(X_train, y_train, epochs = 20, batch_size = 128)

Epoch 1/20
 9/47 [====>.........................] - ETA: 10s - loss: nan - mae: nan - mse: nan                           

KeyboardInterrupt: 

In [ ]:
#保存模型
model_path = f'..//Model//WheatherLSTM_{device}.h5'
from tensorflow.keras.models import save_model
regressor.save(model_path)
print('Model Saved')

Model Saved


In [ ]:
#載入模型
model = load_model(model_path, compile=False)
print('Model Loaded Successfully')

Model Loaded Successfully


In [ ]:
def SeqNumber2Datetime(seq_number: int) -> datetime:
    seq_number = str(seq_number)
    datetime_str = seq_number[:12]
    return datetime.strptime(datetime_str, "%Y%m%d%H%M") 

def Datetime2SeqNumber(dt: datetime, device_id) -> int:
    base_seq_number = dt.strftime("%Y%m%d%H%M")
    seq_number_with_device = f"{base_seq_number}{device_id:02}"
    return seq_number_with_device

In [ ]:
SourceData = pd.read_csv(f"..//Data//MergedSorted//{device}_Merged_Sorted.csv")
PredictedData = SourceData.copy()

TestData = pd.read_csv('..//Data/TestData//upload(no answer).csv')

TestData = TestData[TestData['序號'] % 100 == int(device[1:])]

to_predict_sequmber = TestData['序號'].to_list()

# 預測的資料 的 index
indices_1 = PredictedData[PredictedData['SeqNumber'].isin(to_predict_sequmber)][to_predict_features_model_1].index.to_list()

In [ ]:
index_min = min(indices_1) - n_timesteps
index_max = max(indices_1)

indices_2 = PredictedData.loc[index_min:index_max][to_predict_features_model_1].index.tolist()

# 找出有 NaN 的 row
rows_with_na = PredictedData.loc[indices_2, to_predict_features_model_1].isnull().any(axis=1)
rows_with_na_data = PredictedData.loc[indices_2, to_predict_features_model_1][rows_with_na]

# 有 NaN 的 row 的 index
indices_with_na =  rows_with_na_data.index.to_list()
len(indices_with_na)

436

In [ ]:
 # 如果 LookBackNum > 12 選 indices_with_na
 # 其餘選 indices_1
indices_to_use = []
if LookBackNum > 12:
    indices_to_use = indices_with_na
else:
    indices_to_use = indices_1

for index in indices_to_use:
    X = PredictedData.loc[index-LookBackNum : index-1][to_predict_features_model_1].values
    X = X_preprocess_pipe.transform(X)
    X = np.reshape(X,(1, n_timesteps, n_features))
    pred = regressor.predict(X)
    pred = pred[0]
    PredictedData.loc[index, to_predict_features_model_1] = pred

1/1 [==============================] - 0s 54ms/step


In [ ]:
PredictedData.loc[indices_1][['SeqNumber'] + to_predict_features_model_1].to_csv(f'..//Data//PredictedData//Predicted_{device}.csv', index=False)

In [ ]:
TrainData = pd.read_csv(f'..//Data//MergedSorted//{device}_Merged_Sorted.csv')

TrainData.dropna(inplace=True)
TrainData.reset_index(inplace=True)

input_features = to_predict_features_model_1

to_predict_features = [
    'Avg_Power(mW)',
    ]

In [ ]:
# 訓練 迴歸模型
X_train = TrainData[input_features].values
y_train = TrainData[to_predict_features]

X_train = X_preprocess_pipe.fit_transform(X_train)

reg_model = make_pipeline(
    PCA(n_components=7),
    PolynomialFeatures(degree=4),
    LinearRegression()
)

cv_scores = cross_val_score(reg_model, X_train, y_train, cv=10)
cv_scores

array([0.99845798, 0.99973569, 0.99946629, 0.99954257, 0.99951686,
       0.99983082, 0.99993335, 0.99995589, 0.99923646, 0.99934038])

In [ ]:
reg_model.fit(X_train, y_train)

Pipeline(steps=[('pca', PCA(n_components=7)),
                ('polynomialfeatures', PolynomialFeatures(degree=4)),
                ('linearregression', LinearRegression())])

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = reg_model.predict(X_train)
y_pred = y_pred = np.clip(y_pred, 0, None)

print('MSE: ',mean_squared_error(y_train, y_pred))
print('MAE: ',mean_absolute_error(y_train, y_pred))
print('R2:',r2_score(y_train, y_pred))

MSE:  16.925026234728513
MAE:  2.3227711267591697
R2: 0.9999284456154808


In [ ]:
# 預測 發電量
PredictedData = pd.read_csv(f'..//Data//PredictedData//Predicted_{device}.csv')

X_Test = PredictedData[input_features].values
y_pred = reg_model.predict(X_Test)
y_pred = np.clip(y_pred, 0, None)
PredictedData[to_predict_features] = y_pred
PredictedData.to_csv(f'..//Data//PredictedData//Predicted_{device}.csv', index=False)